## Imports

In [1]:
# Enable automatic extension autoreloading
%load_ext autoreload
%autoreload 2

In [6]:
from pathlib import Path

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import datasets
datasets.disable_progress_bar() # Don't show progress datasets bars
from peft import LoraConfig

import sys
sys.path.insert(0, str(Path.cwd().parent.resolve()))
from model import get_model_with_lora

## Config

In [5]:
device = torch.device(
    # f"cuda:{torch.cuda.current_device()}" if torch.cuda.is_available() else "cpu"
    "cuda:5"
)
device

device(type='cuda', index=5)

In [1]:
# from model import get_model

# # Model
# model = get_model(
#     "lmsys/vicuna-7b-v1.3", device,
#     load_in_8bit=True, low_cpu_mem_usage=True,
#     torch_dtype=torch.float16,
# )
# # model.eval()

# memory_usage = model.pretrained_model.get_memory_footprint() / (1024 ** 3)
# print(f"{memory_usage=:.2f} GB")

In [2]:
# def find_all_linear_names(args, model):
#     cls = torch.nn.Linear
#     lora_module_names = set()
#     for name, module in model.named_modules():
#         if isinstance(module, cls):
#             names = name.split('.')
#             lora_module_names.add(names[0] if len(names) == 1 else names[-1])

#     if 'lm_head' in lora_module_names: # needed for 16-bit
#         lora_module_names.remove('lm_head')

#     return list(lora_module_names)

# modules = find_all_linear_names(None, model)
# print(modules)

In [9]:
# Model
model_name = "lmsys/vicuna-7b-v1.3"

modules = ['down_proj', 'q_proj', 'v_proj', 'o_proj', 'up_proj', 'k_proj', 'gate_proj', 'summary']

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
# )
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_model_with_lora(
    model_name, device, lora_config,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
)
# model.eval()

memory_usage = model.pretrained_model.get_memory_footprint() / (1024 ** 3)
print(f"{memory_usage=:.2f} GB")

Loading policy model...

kwargs={'low_cpu_mem_usage': True, 'torch_dtype': torch.float16}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/fsx/home-augustas/anaconda3/envs/elk/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Loaded subject model with 6,778,396,673 parameters.
Number of trainable params 39,981,057 parameters.
Model dtype: torch.float32

memory_usage=7.19 GB


In [17]:
# for name, p in model.named_parameters():
#     if p.requires_grad:
#         print(".".join(name.split(".")[4:]))
#         print(p.shape)
#         print("-"*100)

layers.0.self_attn.q_proj.lora_A.default.weight
torch.Size([16, 4096])
----------------------------------------------------------------------------------------------------
layers.0.self_attn.q_proj.lora_B.default.weight
torch.Size([4096, 16])
----------------------------------------------------------------------------------------------------
layers.0.self_attn.k_proj.lora_A.default.weight
torch.Size([16, 4096])
----------------------------------------------------------------------------------------------------
layers.0.self_attn.k_proj.lora_B.default.weight
torch.Size([4096, 16])
----------------------------------------------------------------------------------------------------
layers.0.self_attn.v_proj.lora_A.default.weight
torch.Size([16, 4096])
----------------------------------------------------------------------------------------------------
layers.0.self_attn.v_proj.lora_B.default.weight
torch.Size([4096, 16])
---------------------------------------------------------------------

In [11]:
model.training

True

In [12]:
# Was:
# 8,392,705
# 0.1244%

# Num trainable parameters
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_parameters = sum(p.numel() for p in model.parameters())
print(f"{num_trainable_params:,d}")
print(f"{num_trainable_params / num_parameters*100:.4f}%")

39,981,057
0.5898%


In [15]:
model

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096, padding_idx=0)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear8bitLt(
                  in_features=4096, out_features=4096, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
               

In [16]:
# for name, p in model.named_parameters():
#     if p.requires_grad:
#         print(name)